In [0]:
%fs ls /Volumes/workspace/bookstore_eng_ass/dataset/orders-raw/

In [0]:
from pyspark.sql.functions import current_timestamp,col
(
    spark.readStream
           .format("cloudFiles")
           .option("cloudFiles.format", "parquet")
           .option("cloudFiles.schemaLocation","/Volumes/workspace/bookstore_eng_ass/checkpoints/orders_raw")
           .load("/Volumes/workspace/bookstore_eng_ass/dataset/orders-raw/")
           .withColumn("source_file", col("_metadata.file_name"))
           .withColumn("src_file_path", col("_metadata.file_path"))
           .withColumn("arrival_time", current_timestamp())
         .writeStream
           .option("checkpointLocation", "/Volumes/workspace/bookstore_eng_ass/checkpoints/orders_raw")
           .outputMode("append")
           .trigger(availableNow=True)
           .table("workspace.bookstore_eng_ass.orders_bronze")
           .awaitTermination()
)

In [0]:
%sql
select * from workspace.bookstore_eng_ass.orders_bronze

In [0]:
(
    spark.read.format("json")
            .load("/Volumes/workspace/bookstore_eng_ass/dataset/customers-json/")
            .createOrReplaceTempView("customers_lookup")
)

In [0]:
(
    spark.readStream
            .table("orders_bronze")
            .createOrReplaceTempView("orders_bronze_tmp")
)

In [0]:
%sql
create or replace temp view orders_enriched_temp_view
as(
select order_id, quantity, o.customer_id, c.profile:first_name as f_name, c.profile:last_name as l_name,
cast(from_unixtime(order_timestamp, 'yyyy-MM-dd HH:mm:ss') as timestamp) order_timestamp, books
from orders_bronze_tmp o
join customers_lookup c
on o.customer_id = c.customer_id
where quantity > 0) 

In [0]:
(
    spark.table("orders_enriched_temp_view")
         .writeStream
         .option("checkpointLocation","/Volumes/workspace/bookstore_eng_ass/checkpoints/orders_silver")
         .outputMode("append")
         .trigger(availableNow=True)
         .table("workspace.bookstore_eng_ass.orders_silver")
         .awaitTermination()
)

In [0]:
%sql
select * from customers_lookup